In [1]:
def HumanInLoop(x):
    print(x)
    s=input()
    return s

In [2]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import json
import random
from http import HTTPStatus
import dashscope
from dashscope.api_entities.dashscope_response import Role

In [3]:
class AQuADataset(Dataset):
    def __init__(self, data_path, max_records=10):
        with open(data_path, 'r') as file:
            data = [json.loads(line) for line in file]
        data = data[:max_records]
        df = pd.DataFrame(data)
        df = df.head(max_records)
        
        def combine(row):
            choices = [f"{chr(65 + i)}: {option}" for i, option in enumerate(row['options'])]
            return f"Question: {row['question']} Choices: {', '.join(choices)}"
        
        self.inputs = df.apply(combine, axis=1).tolist()
        self.labels = [row['correct'] for row in data]
        
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, idx):
        return self.inputs[idx], self.labels[idx]

# 修改数据路径
aqua_data_path = '/data_zju_2/chenkerui/train.json'

# 实例化AQuA数据集
aqua_dataset = AQuADataset(aqua_data_path)

# 创建DataLoader
aqua_dataloader = DataLoader(aqua_dataset, batch_size=10, shuffle=False)

In [4]:
for batch in aqua_dataloader:
    inputs, labels = batch
    print(f"Inputs: {inputs}")
    print(f"Labels: {labels}")
    break

# 以下代码保持不变
dashscope.api_key = "sk-ea4a0e21cc10489f920bbbf3863d1475"

Inputs: ("Question: Two friends plan to walk along a 43-km trail, starting at opposite ends of the trail at the same time. If Friend P's rate is 15% faster than Friend Q's, how many kilometers will Friend P have walked when they pass each other? Choices: A: A)21, B: B)21.5, C: C)22, D: D)22.5, E: E)23", 'Question: In the coordinate plane, points (x, 1) and (5, y) are on line k. If line k passes through the origin and has slope 1/5, then what are the values of x and y respectively? Choices: A: A)4 and 1, B: B)1 and 5, C: C)5 and 1, D: D)3 and 5, E: E)5 and 3', 'Question: For all numbers p and q, the operation @ is defined by p@q = p^2 - pq. If xy ≠ 0, then which of the following can be equal to zero?\nI. x@y\nII. (xy)@y\nIII. x@(x + y) Choices: A: A)II, B: B)I and II, C: C)I and III, D: D)II and III, E: E)All of the above', 'Question: Carl is facing very difficult financial times and can only pay the interest on a $10,000 loan he has taken. The bank charges him a quarterly compound rate

In [6]:
import requests
from bs4 import BeautifulSoup

def search(search_query):
    url = f"https://www.baidu.com/s?wd={search_query}"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    response = requests.get(url, headers=headers)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "html.parser")

    results = soup.find_all('h3', class_='t', limit=5)

    s=""
    for i, result in enumerate(results, 1):
        title = result.get_text()
        link = result.a['href']
        s+=(f"Result {i}: {title}")
        s+=(f"Link: {link}\n")
    return s



In [22]:
def call_with_messages(model_name: str, content: str):
    messages = [{'role': 'user', 'content': content}]
    response = dashscope.Generation.call(
        model_name,  # 使用指定模型
        messages=messages,
        result_format='message',
        max_tokens=300
    )
    if response.status_code == HTTPStatus.OK:
        return response.output.choices[0]['message']['content']
    else:
        print(f"Request id: {response.request_id}, Status code: {response.status_code}, error code: {response.code}, error message: {response.message}")
        return None

def inference(input):
    outputs = call_with_messages('qwen-72b-chat',input)
    # print("outputs:", outputs)
    return outputs

trigger_sentences = [
    "Let's think step by step.",
]


In [8]:
def test_useHumanInTheLoop():
    correct_count = 0
    total_count = 0
    
    for batch in aqua_dataloader:  # 修改为aqua_dataloader
        inputs, labels = batch

        for question, true_label in zip(inputs, labels):
            print(f"true_label:{true_label}")
            trigger_sentence = (trigger_sentences[0])
            prompt=HumanInLoop(question)
            X0=f"I will give a sample: {prompt}. Please solve follow promblem. Q: {question} A: {trigger_sentence}"
            Z = inference(X0)
            final_prompt = f"{X0} {Z} Therefore, among A through E, the answer is(only one alphabet, no need to write the whole word, e.g., 'A' or 'B' or 'C' or 'D' or 'E')"
            print(final_prompt)
            final_answer = inference(final_prompt)
            prediction = None
            for char in final_answer:
                if char.isupper():
                    prediction = char
                    break
            print(f"prediction:{prediction}")

            total_count+=1
            if (prediction==true_label):correct_count+=1

    accuracy = correct_count / total_count if total_count > 0 else 0
    print(f"Accuracy: {accuracy * 100:.2f}%")

test_useHumanInTheLoop()

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.
true_label:E
Question: Two friends plan to walk along a 43-km trail, starting at opposite ends of the trail at the same time. If Friend P's rate is 15% faster than Friend Q's, how many kilometers will Friend P have walked when they pass each other? Choices: A: A)21, B: B)21.5, C: C)22, D: D)22.5, E: E)23
I will give a sample: Let's denote the speed of Friend Q as  𝑄 Q km/h. According to the problem, Friend P walks 15% faster than Friend Q, so Friend P's speed is  1.15 𝑄 1.15Q km/h.  Both friends start walking from opposite ends of a 43 km trail and will meet somewhere along the trail. When they meet, the sum of the distances each has walked will equal the total length of the trail, which is 43 km.  Let  𝑡 t be the time it takes for them to meet. Friend P will have walked  1.15 𝑄 𝑡 1.15Qt kilometers, and Friend Q will have walked  𝑄 𝑡 Qt kilometers.  Since 

In [10]:
def test_useBaidu():
    correct_count = 0
    total_count = 0
    
    for batch in aqua_dataloader:  # 修改为aqua_dataloader
        inputs, labels = batch

        for question, true_label in zip(inputs, labels):
            print(f"true_label:{true_label}")
            prompt=search(question)
            X0=f"Maybe you can first these contents: {prompt}. Please solve follow promblem. Q: {question} A: Let's think step by step. "
            Z = inference(X0)
            final_prompt = f"{X0} {Z} Therefore, among A through E, the answer is(only one alphabet, no need to write the whole word, e.g., 'A' or 'B' or 'C' or 'D' or 'E')"
            print(final_prompt)
            final_answer = inference(final_prompt)
            prediction = None
            for char in final_answer:
                if char.isupper():
                    prediction = char
                    break
            print(f"prediction:{prediction}")

            total_count+=1
            if (prediction==true_label):correct_count+=1

    accuracy = correct_count / total_count if total_count > 0 else 0
    print(f"Accuracy: {accuracy * 100:.2f}%")

test_useBaidu()

true_label:E
Maybe you can first these contents: Result 1: 【电大国开】国家开放大学23春人文英语1-8形考任务【标准...Link: http://www.baidu.com/link?url=tWpYmZfbD6zeZCquiBu0s0wiQ-502Wke5FTww4VCF_iyaxwXzT4vvYrjCZb8Vi-k
Result 2: 2023年浙江省高三名校协作体联考解析Link: http://www.baidu.com/link?url=tWpYmZfbD6zeZCquiBu0sCcMBnUGwaH4AgPyzTidPMvfxL46yQYNwfCwfs-EEBG7lw6Socm9DGxGCWs4EXO-IhSs8jCvfqLHpJ3ra8I5Md36-UYNM55JNA3Kzd05Zh5pljP78WjYxguMb_4nhnu2a4ULSwnAwNRIbQhRCHgrX_Hh2-jsa-b2hhe-YN5V5sqd_1Iu5OE8-mz95en0_APOFRln-rYs35vmR6NReB2G3TyLkva5iLjd14SJrrFzBSKRNirEH5nW1h-qO8Bo3nqLTK
Result 3: 四六级听力真题练习Day15|备考|day|美文_网易订阅Link: http://www.baidu.com/link?url=sYRgV-6UXSoumt4qUsg7BKq55YmT8L-0dqbUpoeJqExAk8JBT_okJ_2lgVygTzHqUhhidSjmGAFvwBMDz3RCi_
Result 4: 编英文对话:两个同学打算一起去旅行,从商量目的地到...Link: http://www.baidu.com/link?url=2zSqYOO5c2tYsLW2V_NpHlzQSr2BQbZt0xsHQkBZw5Riobr32cN1kvTOzqDYoZhWJmfb2pqitNyXXaknjS2h9AA_H_kX6s_ARpdcZou6U8S
Result 5: 国开《人文英语2》形考任务单元自测7答案 - 百度...Link: http://www.baidu.com/link?url=yV4qf-cq33UHa6L8HutBGSSkCyoGnH09t_3RqgAac-2UU

In [8]:
import yaml
def test_yaml():

    with open('database.yaml', 'r') as file:
        data = yaml.safe_load(file)

    for item in data['questions']:
        print(f"Question {item['id']}: {item['question']}")
        print("Choices:")
        for choice in item['choices']:
            for key, value in choice.items():
                print(f"  {key}: {value}")
        print("Solution:")
        print(item['solution'])
        print("\n" + "-"*50 + "\n")

test_yaml()


Question 1: Two friends plan to walk along a 50-km trail, starting at opposite ends of the trail at the same time. If Friend P's rate is 20% faster than Friend Q's, how many kilometers will Friend P have walked when they pass each other?

Choices:
  A: 26
  B: 27
  C: 28
  D: 29
  E: 30
Solution:
1. Let Friend Q's speed be v km/h. Then Friend P's speed is 1.2v km/h.
2. Let d km be the distance Friend Q walks when they meet, so Friend P walks 50 - d km.
3. Since they meet at the same time:
   d / v = (50 - d) / (1.2v)
4. Cross-multiplying gives:
   1.2d = 50 - d implies 2.2d = 50 implies d = 50 / 2.2 ≈ 22.73
5. Therefore, Friend P walks 50 - 22.73 ≈ 27.27 km.
Answer: B: 27


--------------------------------------------------

Question 2: In the coordinate plane, points (x, 2) and (6, y) are on line k. If line k passes through the origin and has slope 2/3, then what are the values of x and y respectively?

Choices:
  A: 4 and 2
  B: 2 and 4
  C: 3 and 2
  D: 6 and 4
  E: 4 and 6
Solution

In [23]:
def similar(s1,s2):
    verification_prompt = f"Is it possible that the solution's method for problems {s1} and {s2} will be similar (regardless of whether they are both multiple-choice questions)? Just reply me with yes or no"
    verification_result = call_with_messages('qwen-14b-chat', verification_prompt)

    if verification_result:
        return "yes" in verification_result.lower()
    else:
        print("Error in getting verification output.")
        return False

import yaml

def search_similar_problem(s):
    with open('database.yaml', 'r') as file:
        data = yaml.safe_load(file)

    result=""
    for item in data['questions']:
        s1=item['question']
        solu=item['solution']
        if (similar(s,s1)==True):
            result+=f"Q:{s1}. A:{solu}. \n"
    
    if (result==""):
        for item in data['questions']:
            s1=item['question']
            solu=item['solution']
            result+=f"Q:{s1}. A:{solu}. \n"
    return result

def test_InMathDatabase():
    correct_count = 0
    total_count = 0
    
    for batch in aqua_dataloader:  # 修改为aqua_dataloader
        inputs, labels = batch

        for question, true_label in zip(inputs, labels):
            print(f"true_label:{true_label}")
            trigger_sentence = (trigger_sentences[0])
            prompt=search_similar_problem(question)
            X0=f"Maybe you can first read these contents: {prompt}. Please solve follow promblem. Q: {question} A: {trigger_sentence}"
            Z = inference(X0)
            final_prompt = f"{X0} {Z} Therefore, among A through E, the answer is(only one alphabet, no need to write the whole word, e.g., 'A' or 'B' or 'C' or 'D' or 'E')"
            print(final_prompt)
            final_answer = inference(final_prompt)
            prediction = None
            for char in final_answer:
                if char.isupper():
                    prediction = char
                    break
            print(f"prediction:{prediction}")

            total_count+=1
            if (prediction==true_label):correct_count+=1

    accuracy = correct_count / total_count if total_count > 0 else 0
    print(f"Accuracy: {accuracy * 100:.2f}%")

test_InMathDatabase()

true_label:E
Maybe you can first read these contents: Q:Two friends plan to walk along a 50-km trail, starting at opposite ends of the trail at the same time. If Friend P's rate is 20% faster than Friend Q's, how many kilometers will Friend P have walked when they pass each other?
. A:1. Let Friend Q's speed be v km/h. Then Friend P's speed is 1.2v km/h.
2. Let d km be the distance Friend Q walks when they meet, so Friend P walks 50 - d km.
3. Since they meet at the same time:
   d / v = (50 - d) / (1.2v)
4. Cross-multiplying gives:
   1.2d = 50 - d implies 2.2d = 50 implies d = 50 / 2.2 ≈ 22.73
5. Therefore, Friend P walks 50 - 22.73 ≈ 27.27 km.
Answer: B: 27
. 
Q:In the coordinate plane, points (x, 2) and (6, y) are on line k. If line k passes through the origin and has slope 2/3, then what are the values of x and y respectively?
. A:1. Given the slope of the line is 2/3, the equation of the line is y = 2/3x.
2. Substituting point (x, 2):
   2 = 2/3x implies x = 3
3. Substituting poi

In [18]:
def test_baseline():
    correct_count = 0
    total_count = 0
    
    for batch in aqua_dataloader:  # 修改为aqua_dataloader
        inputs, labels = batch

        for question, true_label in zip(inputs, labels):
            X0=f"Q: {question} A: "
            Z = inference(X0)
            final_prompt = f"{X0} {Z} Therefore, among A through E, the answer is(only one alphabet, no need to write the whole word, e.g., 'A' or 'B' or 'C' or 'D' or 'E')"
            print(final_prompt)
            final_answer = inference(final_prompt)
            prediction = None
            for char in final_answer:
                if char.isupper():
                    prediction = char
                    break

            total_count+=1
            if (prediction==true_label):correct_count+=1

    accuracy = correct_count / total_count if total_count > 0 else 0
    print(f"Accuracy: {accuracy * 100:.2f}%")

test_baseline()

Q: Question: Two friends plan to walk along a 43-km trail, starting at opposite ends of the trail at the same time. If Friend P's rate is 15% faster than Friend Q's, how many kilometers will Friend P have walked when they pass each other? Choices: A: A)21, B: B)21.5, C: C)22, D: D)22.5, E: E)23 A:  Let's denote Friend Q's speed as \( Q \) km/h and Friend P's speed as \( P = 1.15Q \) km/h (15% faster). Since they start at opposite ends and walk towards each other, their relative speed is the sum of their speeds, which is \( P + Q = 2.15Q \) km/h.

To find out when they meet, we need to determine the time it takes for them to cover the 43 km between them. Let \( t \) be the time in hours. The distance covered by both friends together is \( 2.15Qt \).

Since this distance equals 43 km, we can set up the equation:

\[ 2.15Qt = 43 \]

To find the time when they pass each other, we solve for \( t \):

\[ t = \frac{43}{2.15Q} \]

Now, since Friend P walks at a rate of \( 1.15Q \), the distanc